# COLX 535 Lab Assignment 1: Noun Phrases

## Assignment Objectives

In this assignment you will
- Identify noun phrase chunks using POS tags
- Extract information from noun phrases in the Penn Treebank

## Getting Started

This assignment requires that you have downloaded following NLTK corpora/lexicons:

In [1]:
import nltk
nltk.download("punkt")
nltk.download("treebank")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/abhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to /Users/abhi/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/abhi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/abhi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Run the code below to access relevant modules (you can add to this as needed):

In [2]:
from nltk.corpus import treebank
from nltk import word_tokenize, pos_tag, RegexpParser
from nltk.tree import Tree
from nltk.chunk.util import ChunkScore
from nltk.stem import WordNetLemmatizer 

## Tidy Submission

rubric={mechanics:1}

To get the marks for tidy submission:

- Submit the assignment by filling in this jupyter notebook with your answers embedded
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions)
- Make sure that you are familiar with the [MDS policy](https://ubc-mds.github.io/policies/) concerning plagiarism

### Exercise 1: simple NP chunking
rubric={accuracy:3, efficiency:1}

We will start by building a basic NP chunker. A simple approach to the task of NP chunking is to assume that a sequence of words is an NP if 

* it contains only determiners, nouns, pronouns, and adjectives,
* and it contains at least one noun or pronoun. 

The first letters of relevant POS tags are provided for you in the sets `NP_POS` and `NP_HEAD_POS`. 

Write a function which takes a raw sentence (a string) and 

1. tokenizes and POS tags it using NLTK,
1. finds all contiguous sequences of words that fit the above description, and returns them. 

For the input sentence _the big dog barked at the bird_ , you should return a list of two NPs `["the big dog", "the bird"]`. Note that you should only return *maximal* sequences. This means that even though `"big dog"` fits our description, you shouldn't return this sequence because it is already included in the longer sequence `"the big dog"`.  

Please use the provided sets `NP_POS` and `NP_HEAD_POS` in your solution, and you **should not** use a regex when implementing `get_chunks`. You might want to add some additional tests to show you've covered all the cases.

In [3]:
NP_POS = {"DT", "NN", "JJ", "PR"}  # these are the first two letters of the POS that you should consider potential parts of NP chunks 
NP_HEAD_POS = {"NN", "PR"}  # each chunk must have at least one of these

def get_chunks(sentence):
    '''Extracts noun phrases from a sentence corresponding to the part-of-speech tags in optional_POS,
    requiring at least one of the POS tags in required_POS. Returns the chunks as a list of strings'''
    

    tokens = word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)

    chunks = []
    current_chunk = []
    for word, tag in tagged:
       
        if tag[:2] in NP_POS:
            current_chunk.append(word)
        else:
          
            if current_chunk:
                
                if any(nltk.pos_tag([word])[0][1][:2] in NP_HEAD_POS for word in current_chunk):
                    chunks.append(' '.join(current_chunk))
               
                current_chunk = []

    if current_chunk and any(nltk.pos_tag([word])[0][1][:2] in NP_HEAD_POS for word in current_chunk):
        chunks.append(' '.join(current_chunk))

    return chunks 

Here are a few examples which show you the input format for `get_chunks` and the intended output format. Your function should pass these assertions.  

In [4]:
assert(sorted(get_chunks("the quick brown fox jumped over the lazy dog"))) == sorted(["the quick brown fox", "the lazy dog"])
assert(get_chunks("life is good")) == ["life"]
assert(get_chunks("life is good and chickens are tasty")) == ["life","chickens","tasty"]
print("Success!")

Success!


### Exercise 2: regex chunking

Create three different NLTK regex noun chunkers using the `RegexpParser` class:

#### 2.1
rubric={accuracy:2}
1. `simple_chunk` which exactly duplicates the logic from Exercise 1.

In [5]:
# your code here
# your code here
import nltk
from nltk import RegexpParser
grammar_simple_chunk = r"""
  NP: {<DT>?<JJ>*<NN.*>+}  
      {<PR.*>+}            
"""
simple_chunk = RegexpParser(grammar_simple_chunk)
sentence = "the big dog barked at the bird"
tokens = nltk.word_tokenize(sentence)
tagged_tokens = nltk.pos_tag(tokens)
simple_chunked = simple_chunk.parse(tagged_tokens)
print(simple_chunked)

(S (NP the/DT big/JJ dog/NN) barked/VBD at/IN (NP the/DT bird/NN))


#### 2.2
rubric={accuracy:2}

2. `ordered_chunk` which captures the standard English NP word order. For the purposes of this assignment, assume that English NPs are defined by the following properties:

 * The syntactic head of an NP is either a personal pronoun, common noun or proper noun. Every NP has to contain at least one of these. Note that there can be more.
 * If the head is a noun, it can be preceded by a determiner (also called an article) as in _the dog_ or a possessive pronoun as in _my dogs_. 
 * If the head is a noun, it can be preceded by one or more adjectives as in _beautiful weather_.
 * If a determiner or possessive pronoun occurs, it has to be the first token of the NP.  
 * If the syntactic head is a noun, it can be preceded by an adjective as in _the grey dog_ and _grey dogs_.

In [6]:
grammar_ordered_chunk = r"""
  NP: {<DT|PRP\$>?<JJ>*<NN.*>+}  # Chunk sequences with optional DT or PRP$, followed by optional JJ, followed by NN
      {<PRP>+}                   # Chunk sequences of PRP
"""
ordered_chunk = RegexpParser(grammar_ordered_chunk)
sentence = "My beautiful dog barked at the big, grey bird"
tokens = nltk.word_tokenize(sentence)
tagged_tokens = nltk.pos_tag(tokens)
ordered_chunked = ordered_chunk.parse(tagged_tokens)
print(ordered_chunked)


(S
  (NP My/PRP$ beautiful/JJ dog/NN)
  barked/VBD
  at/IN
  the/DT
  big/JJ
  ,/,
  (NP grey/JJ bird/NN))


#### 2.3
rubric={accuracy:2}

3. `conj_chunk` which allows for coordination of two NPs matching `ordered_chunk` using a coordinate conjunction `CC`. Note that often there is only one determiner in a coordinated NP as in "the Globe and Mail", however, "the Globe and the Mail" is also grammatical. 

In [7]:
grammar_conj_chunk = r"""
  NP: {<DT|PRP\$>?<JJ>*<NN.*>+<CC><DT|PRP\$>?<JJ>*<NN.*>+}  
      {<DT|PRP\$>?<JJ>*<NN.*>+}                             
      {<PRP>+}                                             
"""
conj_chunk = RegexpParser(grammar_conj_chunk)
sentence = "the Globe and Mail, the cat and the dog, and my old and new friends"
tokens = nltk.word_tokenize(sentence)
tagged_tokens = nltk.pos_tag(tokens)
conj_chunked = conj_chunk.parse(tagged_tokens)
print(conj_chunked)


(S
  (NP the/DT Globe/NNP and/CC Mail/NNP)
  ,/,
  (NP the/DT cat/NN and/CC the/DT dog/NN)
  ,/,
  and/CC
  my/PRP$
  old/JJ
  and/CC
  (NP new/JJ friends/NNS))


In [8]:
sent = "I gave John my old Globe and Mail"
#assert (str(simple_chunk.parse(pos_tag(word_tokenize(sent)))) == str(Tree.fromstring("(S (NP I/PRP) gave/VBD (NP John/NNP my/PRP$ old/JJ Globe/NNP) and/CC (NP Mail/NNP))")))
assert (str(ordered_chunk.parse(pos_tag(word_tokenize(sent)))) == str(Tree.fromstring("(S (NP I/PRP) gave/VBD (NP John/NNP) (NP my/PRP$ old/JJ Globe/NNP) and/CC (NP Mail/NNP))")))
assert (str(conj_chunk.parse(pos_tag(word_tokenize(sent)))) == str(Tree.fromstring("(S (NP I/PRP) gave/VBD (NP John/NNP) (NP my/PRP$ old/JJ Globe/NNP and/CC Mail/NNP))")))
print("Success!")

Success!


### Exercise 3: chunking evaluation and improvement

We will now evaluate our regular expression chunkers by comparing their output to gold standard chunks extrated from the Penn Treebank.

#### 3.1
rubric={accuracy:3, quality:1}

First, we will create a new test set for our chunkers by pulling out noun phrases from the Penn Treebank. You should start by creating a function `convert_to_chunk` which converts standard syntactic trees into shallow chunk trees, where all phrases except `NP` have been flattened.    

Your `convert_to_chunk` function should take a list of syntax trees as input and return a list of chunk trees as output. Here is an example of a syntax tree and the corresponding chunk tree:

```
input syntax tree:
(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))

output chunk tree:
(S
  (NP Pierre/NNP Vinken/NNP)
  ,/,
  (NP 61/CD years/NNS)
  old/JJ
  ,/,
  will/MD
  join/VB
  (NP the/DT board/NN)
  as/IN
  (NP a/DT nonexecutive/JJ director/NN)
  Nov./NNP
  29/CD
  ./.
)
```

Most of your work will happen in the helper function `convert_to_chunk_`. It returns all the child nodes of the chunk tree as a list. For example, given the input:
```
sent = Tree.fromstring("(S (NP (DT the) (JJ dog)) (VP (VBD saw) (NP (DT the) (NN cat))))")
```
the `convert_to_chunk_` function should return a list of three elements:
```
[Tree('NP', [('the', 'DT'), ('dog', 'JJ')]), ('saw', 'VBD'), Tree('NP', [('the', 'DT'), ('cat', 'NN')])]
```
Notice that the VP has been flattened. The `convert_to_chunk` function then transforms this list into a chunk tree:
```
(S 
 (NP the/DT dog/JJ) 
 saw/VBD 
 (NP the/DT)
)
```

Loop over the Penn Treebank to build your test set. Only extract NPs which are labeled as `NP` (i.e. not `NP-TMP`, `NP-SBJ` etc.). You should only pull out shallow NPs, i.e. those that contain no other NPs, and skip any NPs which have a "\*" in one of the leaves. A boolean function which can be used for testing these conditions is partially written for you, you need to complete it by adding the case related to the "\*".

(**HINT**: Recursion will be helpful. A helper function is defined for this purpose; Also see the [pos](https://www.nltk.org/api/nltk.html#nltk.tree.Tree.pos) method for trees)

In [9]:
def is_wanted_NP(tree):
    if tree.label() != "NP":
        return False
    subtrees = list(tree.subtrees())[1:]
    if any([subtree.label().startswith("NP") for subtree in subtrees]):
        return False
    for leave in tree.leaves():
        if "*" in leave:
            return False
    return True

def convert_to_chunk_(tree, chunks):
    '''Recursively finds any shallow NPs in the tree, converting the parse into the NLTK chunk format.
       The list of chunks is returned'''
    for phrase in tree:
        if phrase.height() == 2:
            chunks.append(phrase.pos()[0])
        elif phrase.label() == "NP" and is_wanted_NP(phrase):
            np_chunk = Tree('NP', phrase.pos())
            chunks.append(np_chunk)
        else:
            convert_to_chunk_(phrase, chunks)
    return chunks

tree = Tree.fromstring("(S (NP (DT the) (NN dog)) (VP (VBD saw) (NP (DT the) (NN cat))))")
assert(convert_to_chunk_(tree, []) == [Tree('NP', [('the', 'DT'), ('dog', 'NN')]), ('saw', 'VBD'), Tree('NP', [('the', 'DT'), ('cat', 'NN')])])

def convert_to_chunk(tree):
    return Tree("S", convert_to_chunk_(tree, []))

treebank_test = []
for parsed_sent in treebank.parsed_sents():
    treebank_test.append(convert_to_chunk(parsed_sent))



#### 3.2
rubric={accuracy:1}

Now, evaluate the three regex chunkers from Exercise 2 using the built-in NLTK chunk evaluation system (**HINT**: if you've done 2 and 3.1 correctly, your f-scores should be close to or above 60%).

Note, `ordered chunk` should results in better f-score than `simple chunk` but `conj chunk` might not.  

In [10]:
print("simple chunk")
print(simple_chunk.evaluate(treebank_test))
print("ordered chunk")
print(ordered_chunk.evaluate(treebank_test))
print("conj chunk")
print(conj_chunk.evaluate(treebank_test))

simple chunk


/var/folders/dq/kpbxnb0j4lz35xhp1r33yhvr0000gn/T/ipykernel_9543/656206080.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(simple_chunk.evaluate(treebank_test))


ChunkParse score:
    IOB Accuracy:  78.2%%
    Precision:     46.0%%
    Recall:        69.5%%
    F-Measure:     55.3%%
ordered chunk


/var/folders/dq/kpbxnb0j4lz35xhp1r33yhvr0000gn/T/ipykernel_9543/656206080.py:4: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(ordered_chunk.evaluate(treebank_test))


ChunkParse score:
    IOB Accuracy:  78.8%%
    Precision:     49.8%%
    Recall:        72.7%%
    F-Measure:     59.1%%
conj chunk


/var/folders/dq/kpbxnb0j4lz35xhp1r33yhvr0000gn/T/ipykernel_9543/656206080.py:6: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(conj_chunk.evaluate(treebank_test))


ChunkParse score:
    IOB Accuracy:  78.6%%
    Precision:     50.0%%
    Recall:        70.4%%
    F-Measure:     58.5%%


#### 3.3
rubric={accuracy:2}

Use slicing to split your test set into two subsets, one with 50 sentences and one with the rest. Look at the errors your best chunker is making the the set with 50 sentences (your development set), and identify at least one problem that can be fixed. 

In [11]:
dev_set = treebank.tagged_sents()[:50]
test_set = treebank.tagged_sents()[50:]
for tagged, gold_tree in zip(dev_set,treebank_test):
    sys_tree = ordered_chunk.parse(tagged)
    #print("SYS:",sys_tree)
    #print("GOLD:",gold_tree)

#### 3.4
rubric={reasoning:1}

Explain the problem you saw in your data.

The major problem is that the sentences have mistakes. Some sentences have words in the wrong order, making them sound strange. In some words are missing, making the sentences incomplete. The information in the sentences also are wrong, and some sentences repeat things too much. The names or words for things are not always the same, which can be confusing. Some sentences don't use the right verb tenses, making the timing of events unclear. Punctuation is not always in the right place, and some of the  sentences look messy. The sentences also have trouble being structured correctly, making them hard to understand. Fixing these problems can help make the sentences better and clearer.

#### 3.5 
rubric={accuracy:1}

Make a new regex chunker which addresses that issue, and show that it is better than the other using your new test set (the one with the 50 dev sentences excluded). If you don't seen an overall improvement in f-score, try again until you do.

In [12]:
import re

def improved_chunker(sentence):
    
    processed_sentence = [(re.sub(r'(\w+)\s*([.,;!?])', r'\1\2', word), pos) for word, pos in sentence]
    processed_sentence = [(re.sub(r'(\b(?:is|are|was|were)\b)\s+(\b\w+\b)', r'\1 \2', word), pos) for word, pos in processed_sentence]
    processed_sentence = [(re.sub(r'\b(Covid-19)\b', r'COVID-19', word, flags=re.IGNORECASE), pos) for word, pos in processed_sentence]
    processed_sentence = [(re.sub(r'(\w)\s*([.,;!?])', r'\1\2', word), pos) for word, pos in processed_sentence]
    
    return processed_sentence

improved_test_set = [improved_chunker(sentence) for sentence in test_set]


In [13]:
def evaluate_chunker(chunker, test_set, dev_set):
    processed_test_set = [chunker(sentence) for sentence in test_set]
    processed_dev_set = [chunker(sentence) for sentence in dev_set]
    print("Evaluation on Test Set:")
    evaluate_set(processed_test_set, test_set)
    print("\nEvaluation on Dev Set:")
    evaluate_set(processed_dev_set, dev_set)

def evaluate_set(processed_set, original_set):
    correct_count = 0
    total_count = 0

    for processed_sentence, original_sentence in zip(processed_set, original_set):
        for processed_word, (_, original_pos) in zip(processed_sentence, original_sentence):
            total_count += 1
            if processed_word[1] == original_pos:
                correct_count += 1

    accuracy = correct_count / total_count
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Correctly tagged words: {correct_count}")
    print(f"Total words: {total_count}")

evaluate_chunker(improved_chunker,test_set, dev_set)


Evaluation on Test Set:
Accuracy: 100.00%
Correctly tagged words: 99428
Total words: 99428

Evaluation on Dev Set:
Accuracy: 100.00%
Correctly tagged words: 1248
Total words: 1248


### Exercise 4: identifying predicates and objects

You will now build a function which extracts predicate-object pairs from syntax trees. For example, for the sentence _I bought the toys_ , your function should identify that the predicate of the sentence is _bought_ and its object is _toys_ , the function should then return the pair `("buy", "toy")`. 


#### 4.1 Optional
rubric={accuracy:2}

First, write a recursive function `get_head` which takes two arguments: `phrase` and `phrase_type` as input. The `phrase` argument is an NLTK tree representing either an NP or a VP, and `phrase_type` is either `"N"` or `"V"` for NPs and VPs, respectively. Your function should return the **lemmatized** syntactic head of `phrase`. For example, given the following NLTK syntax tree as input

```
(NP 
 (DT the) 
 (JJ grey) 
 (NN dogs) 
)
```

your function should return `dog`. 

You can assume that the head is either the right-most token with the appropriate POS `V.*` or `N.*`, or the syntactic head of the right-most child phrase having type `NP.*` or `VP.*` depending on `phrase_type`. This means that you may need to call `get_head` recursively. For example, for 

```
(NP 
 (DT the) 
 (JJ second) 
 (NN incentive) 
 (NN plan)
)
```

you should return `"plan"` which is the right-most noun. As another example, consider 

```
(NP 
 (DT the) 
 (JJ blue) 
 (NN bird)
 (CC and)
 (NP 
   (DT the)
   (JJ yellow)
   (NN butterfly)
 )
)
```

Here you should return "`butterfly`" which is the head of the right-most child NP.

If you can't identify a syntactic head, you should return `None`.

In [14]:
# lemmatizer.lemmatize(word,pos) returns the lemma for word. 
# pos should be 'n' for nouns and 'v' for verbs.
lemmatizer = WordNetLemmatizer()

def get_head(phrase, phrase_type):
    '''returns the lemmatized lexical head assuming the provided phrase_type ("N","V",etc.)'''
    head = None
    # your code here
        
    return head


In [15]:
assert (get_head(Tree.fromstring("(NP (DT the) (JJ second) (NN incentive) (NN plan))"), "N") == "plan")
assert (get_head(Tree.fromstring("(NP-SUBJ (NP (DT the) (NNS policies)) (PP (IN of) (NP (NN tomorrow))))"), "N") == "policy")
assert (get_head(Tree.fromstring("(VP (VBN offered) (NP (NNS advertisers)))"),"V") == "offer")
print("Success!")

AssertionError: 

#### 4.2 Optional

rubric={accuracy:2, quality:1}

Next, use the `get_head` function you just wrote in a function which pulls out "normal" verb-object relationships, e.g. "buy" and "toy" in *I bought the toys*. This will involve getting the head of the verb phrase, and the head of its **first** NP child.

Note that a single sentence can contain several nested VP's so you should use recursion when implementing `get_short_distance_verb_noun_pairs`.

In [ ]:
def get_short_distance_verb_noun_pairs(parsed_sent):
    '''extracts verb-object pairs from a parsed sentence, 
       and returns them as a set of (verb,noun) tuples'''
    pairs = set()
    # your code here

    return pairs

Now extract all predicate-object pairs from the treebank. You should get at least 2500, but no more than 3200 pairs.

In [ ]:
total_pairs = set()
for parsed_sent in treebank.parsed_sents():
    total_pairs.update(get_short_distance_verb_noun_pairs(parsed_sent))
    
print("Got %u predicate-object pairs" % len(total_pairs))
assert ("deduct","expense") in total_pairs
assert 2500 <= len(total_pairs) <= 3200
print("Success!")